%%html

<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
    
    .ph-header {
        background: linear-gradient(135deg, #DA552F 0%, #FF6154 100%);
        padding: 60px 40px;
        border-radius: 12px;
        text-align: center;
        margin: 30px 0;
        box-shadow: 0 8px 32px rgba(218, 85, 47, 0.2);
    }
    
    .ph-header h1 {
        color: white;
        font-family: 'Inter', sans-serif;
        font-size: 52px;
        font-weight: 700;
        margin: 0 0 12px 0;
        letter-spacing: -0.5px;
    }
    
    .ph-header p {
        color: rgba(255, 255, 255, 0.95);
        font-family: 'Inter', sans-serif;
        font-size: 18px;
        font-weight: 400;
        margin: 0;
        line-height: 1.6;
    }
    
    .info-card {
        background: #f8f9fa;
        border-left: 3px solid #DA552F;
        padding: 16px 20px;
        border-radius: 6px;
        margin: 20px 0;
    }
    
    .info-card h3 {
        color: #DA552F;
        margin: 0 0 12px 0;
        font-size: 16px;
        font-weight: 600;
    }
</style>

<div class="ph-header">
    <h1>🚀 ProductHuntDB</h1>
    <p>Product Hunt GraphQL API Data Sink & Kaggle Dataset Manager</p>
</div>


# 📖 Overview

This notebook demonstrates how to use **ProductHuntDB** to create, update, and manage a comprehensive Product Hunt dataset on Kaggle.

## ✨ Key Features

- 🔍 **Harvest** data from Product Hunt GraphQL API (posts, users, topics, collections, comments, votes)
- 💾 **Store** in optimized SQLite database with normalized schema
- 🔄 **Sync** incrementally with safety margins to avoid data loss
- 📤 **Publish** to Kaggle with automatic versioning
- ✅ **Validate** data with Pydantic v2 type-safe models

<div class="info-card">
    <h3>🎯 What You'll Learn</h3>
    <ul style="margin: 0; padding-left: 20px;">
        <li>Configure ProductHuntDB for Kaggle Notebooks</li>
        <li>Initialize database and verify API connections</li>
        <li>Perform full and incremental data syncs</li>
        <li>Export data to CSV and publish to Kaggle</li>
        <li>Query and analyze Product Hunt data</li>
    </ul>
</div>

**📚 Resources:** [GitHub](https://github.com/wyattowalsh/producthuntdb) • [Product Hunt API Docs](https://api.producthunt.com/v2/docs)


<div class="info-card">
    <h3>📅 Usage Strategy</h3>
    <p><strong>First Run (Initial Extraction):</strong></p>
    <ul style="margin: 0 0 10px 0; padding-left: 20px;">
        <li>Run cell 9 with <code>!producthuntdb sync --full-refresh</code></li>
        <li>This downloads all historical Product Hunt data (may take several hours)</li>
    </ul>
    <p><strong>Scheduled Daily Updates (Kaggle Cron):</strong></p>
    <ul style="margin: 0; padding-left: 20px;">
        <li>Use default <code>!producthuntdb sync</code> (already configured in cell 9)</li>
        <li>Fetches only new data since last run (fast, typically under 5 minutes)</li>
        <li>Schedule: Notebook → Schedule → Daily</li>
    </ul>
</div>


# 1️⃣ Installation & Setup

Install ProductHuntDB and configure the environment. This cell automatically detects whether you're running on Kaggle or locally and uses the appropriate installation method.


In [ ]:
# Install ProductHuntDB with comprehensive error handling
# Works in Kaggle notebooks and standard Python environments
import subprocess
import sys
import os
from pathlib import Path

print("📦 Installing ProductHuntDB and dependencies...")

# Check if we're in a Kaggle environment or standard Python environment
is_kaggle = Path("/kaggle/working").exists()

try:
    if is_kaggle or "pip" in subprocess.run(
        [sys.executable, "-m", "pip", "--version"], 
        capture_output=True, text=True
    ).stdout:
        # Standard pip installation (works in Kaggle and most Python envs)
        try:
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", "-q",
                "git+https://github.com/wyattowalsh/producthuntdb.git"
            ], stderr=subprocess.DEVNULL)
            print("✅ Installed ProductHuntDB from GitHub")
        except subprocess.CalledProcessError:
            # Fallback to PyPI (if published)
            try:
                subprocess.check_call([
                    sys.executable, "-m", "pip", "install", "-q", "producthuntdb"
                ], stderr=subprocess.DEVNULL)
                print("✅ Installed ProductHuntDB from PyPI")
            except subprocess.CalledProcessError as e:
                print("❌ Installation failed. Please install manually.")
                print("   Run: pip install git+https://github.com/wyattowalsh/producthuntdb.git")
                raise RuntimeError("Failed to install ProductHuntDB") from e
        
        # Install additional dependencies for notebook (not in core package)
        print("📦 Installing notebook-specific dependencies...")
        try:
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", "-q",
                "plotly", "kaleido"  # plotly for interactive viz, kaleido for static export
            ], stderr=subprocess.DEVNULL)
            print("✅ Installed plotly and kaleido")
        except subprocess.CalledProcessError:
            print("⚠️  Optional dependencies (plotly) failed to install")
            print("   Visualizations may not work, but core functionality is OK")
    else:
        # For local development with uv or similar package managers
        print("⚠️  Detected non-standard Python environment.")
        print("   If using uv, run: uv sync --group notebook")
        print("   Otherwise, run: pip install git+https://github.com/wyattowalsh/producthuntdb.git plotly")

    # Configure paths
    print("\n🔧 Configuring environment...")
    WORKING_DIR = Path("/kaggle/working") if is_kaggle else Path.cwd()
    os.environ["DB_PATH"] = str(WORKING_DIR / "producthunt.db")
    os.environ["EXPORT_DIR"] = str(WORKING_DIR / "export")

    # Load secrets from Kaggle Secrets or environment
    token_loaded = False
    kaggle_configured = False
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        
        # Get Product Hunt token (required)
        try:
            producthunt_token = user_secrets.get_secret("PRODUCTHUNT_TOKEN")
            os.environ["PRODUCTHUNT_TOKEN"] = producthunt_token
            print("✅ PRODUCTHUNT_TOKEN loaded from Kaggle Secrets")
            token_loaded = True
        except Exception as e:
            print(f"⚠️  PRODUCTHUNT_TOKEN not found in Kaggle Secrets: {e}")
        
        # Get Kaggle publishing credentials (optional)
        try:
            kaggle_username = user_secrets.get_secret("KAGGLE_USERNAME")
            kaggle_key = user_secrets.get_secret("KAGGLE_KEY")
            kaggle_slug = user_secrets.get_secret("KAGGLE_DATASET_SLUG")
            
            os.environ["KAGGLE_USERNAME"] = kaggle_username
            os.environ["KAGGLE_KEY"] = kaggle_key
            os.environ["KAGGLE_DATASET_SLUG"] = kaggle_slug
            
            print("✅ Kaggle publishing credentials loaded from Secrets")
            kaggle_configured = True
        except Exception:
            print("ℹ️  Kaggle publishing credentials not configured (optional)")
            
    except ImportError:
        # Not in Kaggle environment, try environment variables
        print("ℹ️  Not in Kaggle environment, checking environment variables...")
        
        producthunt_token = os.getenv("PRODUCTHUNT_TOKEN")
        if producthunt_token:
            print("✅ PRODUCTHUNT_TOKEN loaded from environment")
            token_loaded = True
        else:
            print("⚠️  PRODUCTHUNT_TOKEN not found in environment")

    # Summary of configuration
    print(f"\n📂 Working directory: {WORKING_DIR}")
    print(f"💾 Database: {os.environ['DB_PATH']}")
    print(f"📤 Export: {os.environ['EXPORT_DIR']}")
    
    if not token_loaded:
        print("\n🚨 CRITICAL: No PRODUCTHUNT_TOKEN configured!")
        print("   → On Kaggle: Add secret in Notebook Settings → Add-ons → Secrets")
        print("   → Secret name: PRODUCTHUNT_TOKEN")
        print("   → Get token at: https://api.producthunt.com/v2/oauth/applications")
        print("   ⚠️  Pipeline will fail without this token!")
    
    if not kaggle_configured:
        print("\nℹ️  Kaggle dataset publishing not configured (optional)")
        print("   To enable, add these secrets:")
        print("   • KAGGLE_USERNAME")
        print("   • KAGGLE_KEY")
        print("   • KAGGLE_DATASET_SLUG")
    
    # Verify key imports work
    print("\n🔍 Verifying installation...")
    try:
        import producthuntdb
        print("✅ producthuntdb module imported successfully")
    except ImportError as e:
        print(f"❌ Failed to import producthuntdb: {e}")
        raise
    
    try:
        import plotly
        print("✅ plotly module imported successfully")
    except ImportError:
        print("⚠️  plotly not available - visualizations will be limited")
    
    print("\n" + "=" * 60)
    if token_loaded:
        print("✅ SETUP COMPLETE - Ready to proceed!")
    else:
        print("⚠️  SETUP INCOMPLETE - Configure PRODUCTHUNT_TOKEN before proceeding")
    print("=" * 60)
        
except Exception as e:
    print(f"\n❌ Setup failed with error: {str(e)}")
    print("   Check your environment configuration and try again.")
    raise

📦 Installing ProductHuntDB...
⚠️  Detected non-standard Python environment.
   If using uv, run: uv sync
   Otherwise, run: pip install git+https://github.com/wyattowalsh/producthuntdb.git

🔧 Configuring environment...
✅ API token loaded from environment

📂 Working directory: /Users/ww/dev/projects/producthuntdb/notebooks
💾 Database: /Users/ww/dev/projects/producthuntdb/notebooks/producthunt.db
📤 Export: /Users/ww/dev/projects/producthuntdb/notebooks/export


## 🔐 Configuration

### Required: Product Hunt API Token

ProductHuntDB requires a Product Hunt API token to access the GraphQL API.

**On Kaggle:**

1. Go to **Notebook Settings** → **Add-ons** → **Secrets**
2. Add secret: `PRODUCTHUNT_TOKEN` = Your API token from [api.producthunt.com](https://api.producthunt.com/v2/oauth/applications)

**For local development:** Create `.env` file with `PRODUCTHUNT_TOKEN=your_token_here`

<div class="info-card">
    <h3>⚠️ Security Note</h3>
    Never commit API tokens to version control! Always use environment variables or Kaggle Secrets.
</div>

### Optional: Kaggle Publishing

To publish datasets to Kaggle, add these additional secrets:

- `KAGGLE_USERNAME` - Your Kaggle username
- `KAGGLE_KEY` - Your Kaggle API key (from kaggle.com/settings)
- `KAGGLE_DATASET_SLUG` - Dataset slug (format: `username/dataset-name`)


## ✅ Configuration Validation

Let's validate your configuration before proceeding with the pipeline.

In [ ]:
# Configuration validation and health check
import os
from pathlib import Path
import sys

print("🔍 Validating configuration...\n")
print("=" * 60)

validation_results = {
    'critical': [],
    'warnings': [],
    'info': []
}

# 1. Check Python environment
print("1️⃣  Python Environment")
print("-" * 60)
python_version = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
print(f"   Python version: {python_version}")
validation_results['info'].append(f"Python {python_version}")

if sys.version_info >= (3, 11):
    print("   ✅ Python version compatible")
else:
    validation_results['warnings'].append("Python <3.11 (recommended 3.11+)")
    print("   ⚠️  Python 3.11+ recommended for best performance")

# 2. Check required environment variables
print("\n2️⃣  Environment Variables")
print("-" * 60)

# Product Hunt Token
ph_token = os.getenv("PRODUCTHUNT_TOKEN")
if ph_token:
    masked_token = ph_token[:8] + "..." + ph_token[-4:] if len(ph_token) > 12 else "***"
    print(f"   PRODUCTHUNT_TOKEN: {masked_token}")
    print("   ✅ API token configured")
else:
    validation_results['critical'].append("PRODUCTHUNT_TOKEN not set")
    print("   ❌ PRODUCTHUNT_TOKEN: Not set")
    print("      → Add in Kaggle Secrets: Settings → Add-ons → Secrets")

# Database path
db_path = Path(os.getenv("DB_PATH", ""))
if db_path:
    print(f"   DB_PATH: {db_path}")
    if db_path.parent.exists():
        print(f"   ✅ Database directory exists")
    else:
        validation_results['warnings'].append(f"Database directory doesn't exist: {db_path.parent}")
        print(f"   ⚠️  Directory doesn't exist (will be created)")
else:
    validation_results['critical'].append("DB_PATH not configured")
    print("   ❌ DB_PATH: Not configured")

# Export directory
export_dir = Path(os.getenv("EXPORT_DIR", ""))
if export_dir:
    print(f"   EXPORT_DIR: {export_dir}")
    if export_dir.exists():
        print("   ✅ Export directory exists")
    else:
        print("   ⚠️  Will be created on first export")
else:
    validation_results['warnings'].append("EXPORT_DIR not configured")

# 3. Check optional Kaggle publishing credentials
print("\n3️⃣  Kaggle Publishing (Optional)")
print("-" * 60)

kaggle_username = os.getenv("KAGGLE_USERNAME")
kaggle_key = os.getenv("KAGGLE_KEY")
kaggle_slug = os.getenv("KAGGLE_DATASET_SLUG")

publishing_enabled = all([kaggle_username, kaggle_key, kaggle_slug])

if publishing_enabled:
    print(f"   KAGGLE_USERNAME: {kaggle_username}")
    print(f"   KAGGLE_KEY: {'*' * 20}")
    print(f"   KAGGLE_DATASET_SLUG: {kaggle_slug}")
    print("   ✅ Publishing enabled")
else:
    print("   ⚠️  Not configured (dataset publishing disabled)")
    print("   → To enable: Add KAGGLE_USERNAME, KAGGLE_KEY, KAGGLE_DATASET_SLUG to Secrets")

# 4. Check disk space
print("\n4️⃣  System Resources")
print("-" * 60)

try:
    import shutil
    working_dir = Path("/kaggle/working") if Path("/kaggle/working").exists() else Path.cwd()
    disk_stats = shutil.disk_usage(working_dir)
    
    free_gb = disk_stats.free / (1024**3)
    total_gb = disk_stats.total / (1024**3)
    used_percent = (disk_stats.used / disk_stats.total) * 100
    
    print(f"   Disk space: {free_gb:.1f} GB free / {total_gb:.1f} GB total ({used_percent:.1f}% used)")
    
    if free_gb < 1:
        validation_results['critical'].append(f"Low disk space: {free_gb:.1f} GB")
        print("   ❌ Low disk space - may cause failures")
    elif free_gb < 5:
        validation_results['warnings'].append(f"Limited disk space: {free_gb:.1f} GB")
        print("   ⚠️  Limited disk space")
    else:
        print("   ✅ Sufficient disk space")
except Exception as e:
    validation_results['warnings'].append(f"Could not check disk space: {e}")
    print(f"   ⚠️  Could not check disk space: {e}")

# 5. Check package installation
print("\n5️⃣  Package Installation")
print("-" * 60)

try:
    import producthuntdb
    version = getattr(producthuntdb, '__version__', 'unknown')
    print(f"   ProductHuntDB version: {version}")
    print("   ✅ Package installed correctly")
except ImportError:
    validation_results['critical'].append("ProductHuntDB not installed")
    print("   ❌ ProductHuntDB not found")
    print("      → Re-run installation cell")

# Optional dependencies for enhanced features
optional_deps = {
    'plotly': 'Interactive visualizations',
    'pyarrow': 'Parquet export format'
}

for package, description in optional_deps.items():
    try:
        __import__(package)
        print(f"   ✅ {package}: Installed ({description})")
    except ImportError:
        validation_results['info'].append(f"Optional: {package} not installed")
        print(f"   ℹ️  {package}: Not installed ({description} disabled)")

# Summary
print("\n" + "=" * 60)
print("📊 VALIDATION SUMMARY")
print("=" * 60)

if validation_results['critical']:
    print("\n🚨 CRITICAL ISSUES (Must fix before proceeding):")
    for issue in validation_results['critical']:
        print(f"   • {issue}")

if validation_results['warnings']:
    print("\n⚠️  WARNINGS (Recommended to address):")
    for warning in validation_results['warnings']:
        print(f"   • {warning}")

if validation_results['info']:
    print("\nℹ️  INFORMATIONAL:")
    for info in validation_results['info']:
        print(f"   • {info}")

# Overall status
if not validation_results['critical']:
    print("\n" + "=" * 60)
    print("✅ CONFIGURATION VALID - Ready to proceed!")
    print("=" * 60)
else:
    print("\n" + "=" * 60)
    print("❌ CONFIGURATION INVALID - Fix critical issues above")
    print("=" * 60)
    print("\n💡 Quick fixes:")
    if "PRODUCTHUNT_TOKEN" in str(validation_results['critical']):
        print("   1. Go to Notebook Settings → Add-ons → Secrets")
        print("   2. Add secret: PRODUCTHUNT_TOKEN = <your_token>")
        print("   3. Get token from: https://api.producthunt.com/v2/oauth/applications")
        print("   4. Re-run this cell to validate")

# 2️⃣ Initialize Database & Verify Connection

Let's initialize the database and verify our API authentication works correctly.


In [ ]:
# Initialize database and verify authentication with error handling
import subprocess
import sys

print("⏱️  Expected runtime: ~10-30 seconds\n")

try:
    # Initialize database
    print("🔧 Initializing database...")
    result = subprocess.run(
        ["producthuntdb", "init"],
        capture_output=True,
        text=True,
        check=False
    )
    
    if result.returncode == 0:
        print("✅ Database initialized successfully")
        print(result.stdout)
    else:
        print("⚠️  Database initialization encountered issues:")
        print(result.stderr)
        if "already exists" in result.stderr.lower():
            print("   (Database may already be initialized - this is usually fine)")
        else:
            raise RuntimeError(f"Database init failed: {result.stderr}")
    
    # Verify API authentication
    print("\n🔐 Verifying API authentication...")
    result = subprocess.run(
        ["producthuntdb", "verify"],
        capture_output=True,
        text=True,
        check=False
    )
    
    if result.returncode == 0:
        print("✅ API authentication verified")
        print(result.stdout)
    else:
        print("❌ API authentication failed:")
        print(result.stderr)
        print("\n💡 Troubleshooting:")
        print("   1. Check your PRODUCTHUNT_TOKEN is valid")
        print("   2. Get a new token at: https://api.producthunt.com/v2/oauth/applications")
        print("   3. Verify the token is correctly set in Kaggle Secrets or environment")
        raise RuntimeError(f"API verification failed: {result.stderr}")
        
except FileNotFoundError:
    print("❌ 'producthuntdb' command not found!")
    print("   The package may not be installed correctly.")
    print("   Try re-running the installation cell above.")
    raise
except Exception as e:
    print(f"\n❌ Initialization failed: {str(e)}")
    print("   Check logs above for specific error details.")
    raise

# 3️⃣ Sync Data from Product Hunt

Sync data from the Product Hunt API. Start with a limited sync to test, then perform a full sync when ready.

**Sync Options:**

- **Full Refresh** - Downloads all historical data (can take several hours)
- **Incremental Update** - Only syncs new/updated data since last run (fast)
- **Limited Sync** - Fetches a specific number of pages (good for testing)

<div class="info-card">
    <h3>💡 Recommended Approach</h3>
    Start with <code>--max-pages 10</code> to test the setup, then run a full sync: <code>producthuntdb sync --full-refresh</code>
</div>


In [ ]:
# Sync data from Product Hunt with comprehensive error handling
# Configure sync strategy based on your needs
import subprocess
import sys
from datetime import datetime
from pathlib import Path

print("⏱️  Expected runtime:")
print("   • Full refresh: 2-4 hours (first run)")
print("   • Incremental: 3-5 minutes (daily updates)")
print("   • Limited test: 1-2 minutes (--max-pages 10)\n")

# Track sync timing
start_time = datetime.now()
print(f"🚀 Starting sync at {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

try:
    # 🎯 FOR FIRST RUN: Uncomment this line to get all historical data
    # sync_command = ["producthuntdb", "sync", "--full-refresh"]
    
    # 🔄 FOR SCHEDULED DAILY UPDATES: Use this (default, fast incremental updates)
    sync_command = ["producthuntdb", "sync"]
    
    # 🧪 FOR TESTING: Limit to a few pages (uncomment to use)
    # sync_command = ["producthuntdb", "sync", "--max-pages", "10"]
    
    # 📊 POSTS ONLY: Skip topics and collections (faster, uncomment to use)
    # sync_command = ["producthuntdb", "sync", "--posts-only"]
    
    print(f"📡 Running command: {' '.join(sync_command)}\n")
    
    result = subprocess.run(
        sync_command,
        capture_output=True,
        text=True,
        check=False,
        timeout=14400  # 4-hour timeout (Kaggle limit is 12 hours)
    )
    
    # Calculate elapsed time
    end_time = datetime.now()
    elapsed = end_time - start_time
    
    if result.returncode == 0:
        print(result.stdout)
        print(f"\n✅ Sync completed successfully in {elapsed.total_seconds():.1f} seconds")
        print(f"   ({elapsed.total_seconds() / 60:.1f} minutes)")
    else:
        print("⚠️  Sync encountered errors:")
        print(result.stderr)
        
        # Provide context-specific troubleshooting
        if "rate limit" in result.stderr.lower():
            print("\n💡 Rate Limit Hit - Troubleshooting:")
            print("   • The API has rate limits that reset periodically")
            print("   • Built-in retry logic will handle this automatically")
            print("   • For faster testing, use --max-pages option")
            print("   • Consider running sync during off-peak hours")
        elif "timeout" in result.stderr.lower():
            print("\n💡 Timeout - Troubleshooting:")
            print("   • Full refresh can take several hours")
            print("   • Use incremental sync for daily updates")
            print("   • Data collected before timeout is safely stored")
            print("   • Re-run to continue from where it left off")
        elif "authentication" in result.stderr.lower() or "token" in result.stderr.lower():
            print("\n💡 Authentication Error - Troubleshooting:")
            print("   • Verify PRODUCTHUNT_TOKEN is set correctly")
            print("   • Token may have expired - get new one from api.producthunt.com")
            print("   • Check Kaggle Secrets configuration")
        else:
            print("\n💡 General Troubleshooting:")
            print("   • Check database file is not corrupted")
            print("   • Verify sufficient disk space available")
            print("   • Review full error message above")
            print("   • Try running 'producthuntdb status' to check database state")
        
        # Don't raise if partial success (some data may have been synced)
        if "error" in result.stderr.lower() and result.stdout:
            print("\n⚠️  Partial sync completed - some data was saved before error")
        else:
            raise RuntimeError(f"Sync failed: {result.stderr}")
    
    # Save sync timing for performance monitoring
    try:
        with open("/kaggle/working/sync_history.txt" if Path("/kaggle/working").exists() else "sync_history.txt", "a") as f:
            f.write(f"{start_time.isoformat()},{elapsed.total_seconds()},{result.returncode}\n")
    except Exception:
        pass  # Non-critical if we can't save timing data
        
except subprocess.TimeoutExpired:
    end_time = datetime.now()
    elapsed = end_time - start_time
    print(f"\n⏱️  Sync timed out after {elapsed.total_seconds() / 3600:.1f} hours")
    print("   Data collected up to this point has been saved to the database.")
    print("   You can re-run sync to continue where it left off.")
    print("\n💡 To avoid timeouts:")
    print("   • Use incremental sync instead of --full-refresh")
    print("   • Run during off-peak hours")
    print("   • Consider splitting into smaller batches with --max-pages")
except FileNotFoundError:
    print("❌ 'producthuntdb' command not found!")
    print("   Re-run the installation cell to fix this.")
    raise
except Exception as e:
    end_time = datetime.now()
    elapsed = end_time - start_time
    print(f"\n❌ Sync failed after {elapsed.total_seconds():.1f} seconds")
    print(f"   Error: {str(e)}")
    raise

# 4️⃣ Database Statistics & Status

Let's examine what we've collected and view key statistics about the database.


In [ ]:
# View database statistics
!producthuntdb status

# 📋 Data Quality Validation

Let's validate the data quality with comprehensive checks for completeness, freshness, and integrity.

In [ ]:
# Comprehensive data quality validation
import pandas as pd
import sqlite3
import os
from pathlib import Path
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("⏱️  Expected runtime: ~10-15 seconds\n")
print("🔍 Running data quality checks...\n")

try:
    db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))
    conn = sqlite3.connect(db_path)
    
    # 1. Data Completeness Check
    print("=" * 60)
    print("1️⃣  DATA COMPLETENESS")
    print("=" * 60)
    
    completeness = pd.read_sql_query(
        """
        SELECT 
            'Posts' as entity,
            COUNT(*) as total_records,
            SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) as missing_name,
            SUM(CASE WHEN tagline IS NULL THEN 1 ELSE 0 END) as missing_tagline,
            SUM(CASE WHEN featured_at IS NULL THEN 1 ELSE 0 END) as missing_date,
            ROUND(100.0 * SUM(CASE WHEN name IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as name_completeness
        FROM post_row
        UNION ALL
        SELECT 
            'Users',
            COUNT(*),
            SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END),
            SUM(CASE WHEN username IS NULL THEN 1 ELSE 0 END),
            SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END),
            ROUND(100.0 * SUM(CASE WHEN name IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 2)
        FROM user_row
        UNION ALL
        SELECT 
            'Topics',
            COUNT(*),
            SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END),
            SUM(CASE WHEN slug IS NULL THEN 1 ELSE 0 END),
            0,
            ROUND(100.0 * SUM(CASE WHEN name IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 2)
        FROM topic_row
    """,
        conn,
    )
    display(completeness)
    
    # Flag critical issues
    critical_issues = []
    for _, row in completeness.iterrows():
        if row['name_completeness'] < 95:
            critical_issues.append(f"⚠️  {row['entity']}: Only {row['name_completeness']}% name completeness")
    
    if critical_issues:
        print("\n🚨 CRITICAL ISSUES DETECTED:")
        for issue in critical_issues:
            print(f"   {issue}")
    else:
        print("\n✅ All entities have >95% completeness")
    
    # 2. Data Freshness Check
    print("\n" + "=" * 60)
    print("2️⃣  DATA FRESHNESS")
    print("=" * 60)
    
    freshness = pd.read_sql_query(
        """
        SELECT 
            MAX(DATE(featured_at)) as latest_post,
            MIN(DATE(featured_at)) as earliest_post,
            COUNT(*) as total_posts,
            JULIANDAY('now') - JULIANDAY(MAX(featured_at)) as days_since_latest
        FROM post_row
        WHERE featured_at IS NOT NULL
    """,
        conn,
    )
    
    if not freshness.empty:
        display(freshness)
        
        days_stale = freshness['days_since_latest'].iloc[0]
        if days_stale > 2:
            print(f"\n⚠️  Data is {days_stale:.1f} days old - consider running sync")
        else:
            print(f"\n✅ Data is fresh ({days_stale:.1f} days old)")
    
    # 3. Duplicate Detection
    print("\n" + "=" * 60)
    print("3️⃣  DUPLICATE DETECTION")
    print("=" * 60)
    
    duplicates = pd.read_sql_query(
        """
        SELECT 
            'Posts' as entity,
            COUNT(*) - COUNT(DISTINCT id) as duplicate_ids,
            COUNT(*) - COUNT(DISTINCT name) as duplicate_names
        FROM post_row
        UNION ALL
        SELECT 
            'Users',
            COUNT(*) - COUNT(DISTINCT id),
            COUNT(*) - COUNT(DISTINCT username)
        FROM user_row
        UNION ALL
        SELECT 
            'Topics',
            COUNT(*) - COUNT(DISTINCT id),
            COUNT(*) - COUNT(DISTINCT slug)
        FROM topic_row
    """,
        conn,
    )
    display(duplicates)
    
    total_duplicates = duplicates['duplicate_ids'].sum()
    if total_duplicates > 0:
        print(f"\n⚠️  Found {total_duplicates} duplicate IDs - may need data cleaning")
    else:
        print("\n✅ No duplicate IDs detected")
    
    # 4. Relationship Integrity
    print("\n" + "=" * 60)
    print("4️⃣  RELATIONSHIP INTEGRITY")
    print("=" * 60)
    
    integrity = pd.read_sql_query(
        """
        SELECT 
            'Orphaned Comments' as check_type,
            COUNT(*) as count
        FROM comment_row c
        WHERE NOT EXISTS (SELECT 1 FROM post_row p WHERE p.id = c.post_id)
        UNION ALL
        SELECT 
            'Orphaned Votes',
            COUNT(*)
        FROM vote_row v
        WHERE NOT EXISTS (SELECT 1 FROM post_row p WHERE p.id = v.post_id)
        UNION ALL
        SELECT 
            'Orphaned Maker Links',
            COUNT(*)
        FROM maker_post_link mpl
        WHERE NOT EXISTS (SELECT 1 FROM post_row p WHERE p.id = mpl.post_id)
           OR NOT EXISTS (SELECT 1 FROM user_row u WHERE u.id = mpl.maker_id)
    """,
        conn,
    )
    display(integrity)
    
    integrity_issues = integrity[integrity['count'] > 0]
    if len(integrity_issues) > 0:
        print(f"\n⚠️  Found {len(integrity_issues)} integrity issues")
    else:
        print("\n✅ All relationships are valid")
    
    # 5. Data Distribution Analysis
    print("\n" + "=" * 60)
    print("5️⃣  DATA DISTRIBUTION")
    print("=" * 60)
    
    distribution = pd.read_sql_query(
        """
        SELECT 
            strftime('%Y-%m', featured_at) as month,
            COUNT(*) as posts,
            AVG(votes_count) as avg_votes,
            MAX(votes_count) as max_votes
        FROM post_row
        WHERE featured_at >= date('now', '-12 months')
        GROUP BY month
        ORDER BY month DESC
        LIMIT 12
    """,
        conn,
    )
    
    if not distribution.empty:
        display(distribution)
        
        # Check for anomalies
        if distribution['posts'].std() / distribution['posts'].mean() > 0.5:
            print("\n⚠️  High variance in monthly post counts - data may be incomplete")
        else:
            print("\n✅ Consistent data distribution")
    
    # 6. Quality Score Summary
    print("\n" + "=" * 60)
    print("📊 OVERALL QUALITY SCORE")
    print("=" * 60)
    
    quality_metrics = {
        'Completeness': 100 - (completeness['missing_name'].sum() / completeness['total_records'].sum() * 100),
        'Freshness': max(0, 100 - (days_stale * 10)),
        'Uniqueness': 100 - (total_duplicates / completeness['total_records'].sum() * 100) if completeness['total_records'].sum() > 0 else 100,
        'Integrity': 100 - (integrity['count'].sum() / completeness['total_records'].sum() * 100) if completeness['total_records'].sum() > 0 else 100
    }
    
    quality_df = pd.DataFrame(list(quality_metrics.items()), columns=['Metric', 'Score'])
    quality_df['Score'] = quality_df['Score'].round(2)
    display(quality_df)
    
    overall_score = sum(quality_metrics.values()) / len(quality_metrics)
    print(f"\n🎯 Overall Data Quality Score: {overall_score:.1f}/100")
    
    if overall_score >= 90:
        print("   ✅ EXCELLENT - Data is production-ready")
    elif overall_score >= 75:
        print("   ⚠️  GOOD - Minor issues detected, consider review")
    elif overall_score >= 60:
        print("   ⚠️  FAIR - Notable issues, recommend data cleaning")
    else:
        print("   🚨 POOR - Critical issues, data quality needs attention")
    
    # Create quality score visualization
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=quality_df['Metric'],
        y=quality_df['Score'],
        marker_color=['#2ecc71' if s >= 90 else '#f39c12' if s >= 75 else '#e74c3c' for s in quality_df['Score']],
        text=quality_df['Score'].round(1),
        textposition='auto',
    ))
    
    fig.update_layout(
        title='Data Quality Metrics Dashboard',
        xaxis_title='Quality Dimension',
        yaxis_title='Score (0-100)',
        yaxis_range=[0, 105],
        showlegend=False,
        hovermode='x'
    )
    
    fig.show()
    
    conn.close()
    print("\n✅ Data quality validation complete!")
    
except Exception as e:
    print(f"\n❌ Validation failed: {str(e)}")
    raise
finally:
    try:
        conn.close()
    except:
        pass

# 5️⃣ Query & Analyze Data

Let's explore the data we've collected with some SQL queries using pandas.


In [ ]:
# Query and analyze data with memory-efficient patterns
import pandas as pd
import sqlite3
import os
from pathlib import Path

print("⏱️  Expected runtime: ~5-10 seconds\n")

try:
    # Connect to the database
    db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))
    
    if not db_path.exists():
        print("❌ Database not found! Run the sync cell first.")
        raise FileNotFoundError(f"Database not found at {db_path}")
    
    conn = sqlite3.connect(db_path)
    
    # Use chunksize for large queries to avoid memory issues
    CHUNK_SIZE = 10000
    
    print("🏆 Top 10 Products by Votes\n")
    top_posts = pd.read_sql_query(
        """
        SELECT 
            name,
            tagline,
            votes_count,
            comments_count,
            DATE(featured_at) as featured_date,
            url
        FROM post_row
        WHERE votes_count IS NOT NULL
        ORDER BY votes_count DESC
        LIMIT 10
    """,
        conn,
    )
    
    if top_posts.empty:
        print("⚠️  No posts found. Run sync to populate database.")
    else:
        # Format for better display
        top_posts['featured_date'] = pd.to_datetime(top_posts['featured_date'])
        display(top_posts)
    
    print("\n👤 Top 10 Most Active Makers\n")
    active_makers = pd.read_sql_query(
        """
        SELECT 
            u.name,
            u.username,
            COUNT(DISTINCT mpl.post_id) as products_made,
            u.url,
            u.headline
        FROM user_row u
        JOIN maker_post_link mpl ON u.id = mpl.maker_id
        GROUP BY u.id
        ORDER BY products_made DESC
        LIMIT 10
    """,
        conn,
    )
    
    if active_makers.empty:
        print("⚠️  No maker data found. Ensure sync completed successfully.")
    else:
        display(active_makers)
    
    print("\n🏷️  Top 10 Popular Topics\n")
    popular_topics = pd.read_sql_query(
        """
        SELECT 
            t.name,
            t.slug,
            COUNT(DISTINCT ptl.post_id) as product_count,
            t.description
        FROM topic_row t
        JOIN post_topic_link ptl ON t.id = ptl.topic_id
        GROUP BY t.id
        ORDER BY product_count DESC
        LIMIT 10
    """,
        conn,
    )
    
    if popular_topics.empty:
        print("⚠️  No topic data found. Topics may not have been synced.")
    else:
        display(popular_topics)
    
    # Additional analytics: Recent high-engagement products
    print("\n🔥 Recently Launched High-Engagement Products (Last 30 Days)\n")
    recent_hot = pd.read_sql_query(
        """
        SELECT 
            name,
            tagline,
            votes_count,
            comments_count,
            DATE(featured_at) as launch_date,
            (votes_count + comments_count * 2) as engagement_score
        FROM post_row
        WHERE featured_at >= date('now', '-30 days')
            AND votes_count > 0
        ORDER BY engagement_score DESC
        LIMIT 10
    """,
        conn,
    )
    
    if not recent_hot.empty:
        recent_hot['launch_date'] = pd.to_datetime(recent_hot['launch_date'])
        display(recent_hot)
    else:
        print("⚠️  No recent products found. Database may need more data.")
    
    # Database statistics
    print("\n📊 Database Statistics\n")
    stats = pd.read_sql_query(
        """
        SELECT 
            'Posts' as table_name, COUNT(*) as row_count FROM post_row
        UNION ALL
        SELECT 'Users', COUNT(*) FROM user_row
        UNION ALL
        SELECT 'Topics', COUNT(*) FROM topic_row
        UNION ALL
        SELECT 'Comments', COUNT(*) FROM comment_row
        UNION ALL
        SELECT 'Collections', COUNT(*) FROM collection_row
        UNION ALL
        SELECT 'Votes', COUNT(*) FROM vote_row
    """,
        conn,
    )
    display(stats)
    
    conn.close()
    print("\n✅ Query analysis complete!")
    
except sqlite3.Error as e:
    print(f"❌ Database error: {str(e)}")
    print("\n💡 Troubleshooting:")
    print("   • Database may be locked by another process")
    print("   • Try closing other connections to the database")
    print("   • Run 'producthuntdb status' to check database health")
    raise
except pd.errors.DatabaseError as e:
    print(f"❌ Query error: {str(e)}")
    print("\n💡 Troubleshooting:")
    print("   • Database schema may be outdated")
    print("   • Try running 'producthuntdb upgrade head' to apply migrations")
    raise
except MemoryError:
    print("❌ Out of memory!")
    print("\n💡 Troubleshooting:")
    print("   • Query returned too much data")
    print("   • Use LIMIT clauses to reduce result size")
    print("   • Consider using chunked reading for large datasets")
    raise
except Exception as e:
    print(f"❌ Query failed: {str(e)}")
    raise
finally:
    # Ensure connection is closed
    try:
        conn.close()
    except:
        pass

## 🔬 Advanced Query Examples

Explore Product Hunt data with sophisticated analytical queries for deeper insights.

In [ ]:
# Advanced analytical queries for Product Hunt insights
import pandas as pd
import sqlite3
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("⏱️  Expected runtime: ~10-15 seconds\n")
print("🔬 Running advanced analytics...\n")

try:
    db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))
    if not db_path.exists():
        print("❌ Database not found! Run sync first.")
        raise FileNotFoundError(f"Database not found at {db_path}")
    
    conn = sqlite3.connect(db_path)
    
    # 1. Maker Network Analysis - Find prolific makers and their collaborators
    print("=" * 60)
    print("1️⃣  MAKER NETWORK ANALYSIS")
    print("=" * 60)
    
    maker_network = pd.read_sql_query(
        """
        WITH maker_stats AS (
            SELECT 
                u.id,
                u.name,
                u.username,
                COUNT(DISTINCT mpl.post_id) as products_made,
                AVG(p.votes_count) as avg_votes_per_product,
                MAX(p.votes_count) as best_product_votes,
                COUNT(DISTINCT CASE WHEN p.featured_at >= date('now', '-90 days') THEN mpl.post_id END) as recent_products
            FROM user_row u
            JOIN maker_post_link mpl ON u.id = mpl.maker_id
            JOIN post_row p ON mpl.post_id = p.id
            GROUP BY u.id
            HAVING products_made >= 3
        )
        SELECT *
        FROM maker_stats
        ORDER BY products_made DESC, avg_votes_per_product DESC
        LIMIT 20
    """,
        conn
    )
    
    if not maker_network.empty:
        print(f"\n📊 Top 20 Prolific Makers (3+ products):\n")
        maker_network['avg_votes_per_product'] = maker_network['avg_votes_per_product'].round(1)
        display(maker_network)
        
        # Visualize maker productivity
        fig1 = px.scatter(
            maker_network,
            x='products_made',
            y='avg_votes_per_product',
            size='best_product_votes',
            hover_data=['name', 'username', 'recent_products'],
            title='Maker Productivity: Quantity vs Quality',
            labels={
                'products_made': 'Number of Products',
                'avg_votes_per_product': 'Average Votes per Product'
            },
            color='recent_products',
            color_continuous_scale='Viridis'
        )
        fig1.show()
    else:
        print("⚠️  Insufficient data for maker network analysis")
    
    # 2. Topic Evolution - Trending topics over time
    print("\n" + "=" * 60)
    print("2️⃣  TOPIC EVOLUTION & TRENDS")
    print("=" * 60)
    
    topic_trends = pd.read_sql_query(
        """
        SELECT 
            t.name as topic,
            strftime('%Y-%m', p.featured_at) as month,
            COUNT(DISTINCT p.id) as product_count,
            AVG(p.votes_count) as avg_votes
        FROM topic_row t
        JOIN post_topic_link ptl ON t.id = ptl.topic_id
        JOIN post_row p ON ptl.post_id = p.id
        WHERE p.featured_at >= date('now', '-12 months')
            AND t.name IN (
                SELECT t2.name 
                FROM topic_row t2
                JOIN post_topic_link ptl2 ON t2.id = ptl2.topic_id
                GROUP BY t2.id
                ORDER BY COUNT(DISTINCT ptl2.post_id) DESC
                LIMIT 8
            )
        GROUP BY t.name, month
        ORDER BY month DESC, product_count DESC
    """,
        conn
    )
    
    if not topic_trends.empty:
        # Pivot for heatmap
        topic_pivot = topic_trends.pivot(index='topic', columns='month', values='product_count')
        topic_pivot = topic_pivot.fillna(0)
        
        fig2 = go.Figure(data=go.Heatmap(
            z=topic_pivot.values,
            x=topic_pivot.columns,
            y=topic_pivot.index,
            colorscale='YlOrRd',
            hoverongaps=False
        ))
        
        fig2.update_layout(
            title='Topic Popularity Heatmap (Last 12 Months)',
            xaxis_title='Month',
            yaxis_title='Topic',
            height=500
        )
        fig2.show()
        
        print(f"\n📈 Topic trends data:")
        display(topic_trends.head(20))
    else:
        print("⚠️  Insufficient data for topic trends")
    
    # 3. Launch Timing Analysis - Best days and times to launch
    print("\n" + "=" * 60)
    print("3️⃣  OPTIMAL LAUNCH TIMING")
    print("=" * 60)
    
    launch_timing = pd.read_sql_query(
        """
        SELECT 
            CASE CAST(strftime('%w', featured_at) AS INTEGER)
                WHEN 0 THEN 'Sunday'
                WHEN 1 THEN 'Monday'
                WHEN 2 THEN 'Tuesday'
                WHEN 3 THEN 'Wednesday'
                WHEN 4 THEN 'Thursday'
                WHEN 5 THEN 'Friday'
                WHEN 6 THEN 'Saturday'
            END as day_of_week,
            CAST(strftime('%w', featured_at) AS INTEGER) as day_num,
            CAST(strftime('%H', featured_at) AS INTEGER) as hour,
            COUNT(*) as launch_count,
            AVG(votes_count) as avg_votes,
            AVG(comments_count) as avg_comments,
            AVG(votes_count + comments_count * 2) as avg_engagement
        FROM post_row
        WHERE featured_at IS NOT NULL
            AND featured_at >= date('now', '-180 days')
        GROUP BY day_of_week, day_num, hour
        ORDER BY day_num, hour
    """,
        conn
    )
    
    if not launch_timing.empty:
        # Create heatmap of best launch times
        timing_pivot = launch_timing.pivot_table(
            index='hour',
            columns='day_of_week',
            values='avg_engagement',
            fill_value=0
        )
        
        # Reorder columns to standard week order
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        timing_pivot = timing_pivot[[col for col in day_order if col in timing_pivot.columns]]
        
        fig3 = go.Figure(data=go.Heatmap(
            z=timing_pivot.values,
            x=timing_pivot.columns,
            y=timing_pivot.index,
            colorscale='RdYlGn',
            hoverongaps=False
        ))
        
        fig3.update_layout(
            title='Best Launch Times (Engagement Score = Votes + Comments×2)',
            xaxis_title='Day of Week',
            yaxis_title='Hour (UTC)',
            height=600
        )
        fig3.show()
        
        # Find top 5 best time slots
        best_times = launch_timing.nlargest(5, 'avg_engagement')[
            ['day_of_week', 'hour', 'avg_engagement', 'launch_count']
        ]
        print("\n🎯 Top 5 Launch Time Slots:")
        display(best_times)
    else:
        print("⚠️  Insufficient data for timing analysis")
    
    # 4. Product Success Patterns - What makes products successful?
    print("\n" + "=" * 60)
    print("4️⃣  SUCCESS PATTERN ANALYSIS")
    print("=" * 60)
    
    success_patterns = pd.read_sql_query(
        """
        WITH product_metrics AS (
            SELECT 
                p.id,
                p.name,
                p.votes_count,
                p.comments_count,
                LENGTH(p.tagline) as tagline_length,
                LENGTH(p.description) as description_length,
                COUNT(DISTINCT mpl.maker_id) as maker_count,
                COUNT(DISTINCT ptl.topic_id) as topic_count,
                CASE 
                    WHEN p.votes_count >= (SELECT PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY votes_count) FROM post_row)
                    THEN 'Top 10%'
                    WHEN p.votes_count >= (SELECT PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY votes_count) FROM post_row)
                    THEN 'Top 25%'
                    WHEN p.votes_count >= (SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY votes_count) FROM post_row)
                    THEN 'Top 50%'
                    ELSE 'Bottom 50%'
                END as success_tier
            FROM post_row p
            LEFT JOIN maker_post_link mpl ON p.id = mpl.post_id
            LEFT JOIN post_topic_link ptl ON p.id = ptl.post_id
            WHERE p.votes_count > 0
            GROUP BY p.id
        )
        SELECT 
            success_tier,
            COUNT(*) as product_count,
            AVG(votes_count) as avg_votes,
            AVG(comments_count) as avg_comments,
            AVG(tagline_length) as avg_tagline_length,
            AVG(maker_count) as avg_makers,
            AVG(topic_count) as avg_topics
        FROM product_metrics
        GROUP BY success_tier
        ORDER BY 
            CASE success_tier
                WHEN 'Top 10%' THEN 1
                WHEN 'Top 25%' THEN 2
                WHEN 'Top 50%' THEN 3
                ELSE 4
            END
    """,
        conn
    )
    
    if not success_patterns.empty:
        print("\n🎯 Success Patterns by Performance Tier:\n")
        display(success_patterns.round(2))
        
        # Visualize patterns
        fig4 = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Makers', 'Average Topics', 'Tagline Length', 'Comments Engagement'),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "bar"}, {"type": "bar"}]]
        )
        
        fig4.add_trace(go.Bar(x=success_patterns['success_tier'], y=success_patterns['avg_makers'], 
                              name='Makers', marker_color='lightblue'), row=1, col=1)
        fig4.add_trace(go.Bar(x=success_patterns['success_tier'], y=success_patterns['avg_topics'], 
                              name='Topics', marker_color='lightgreen'), row=1, col=2)
        fig4.add_trace(go.Bar(x=success_patterns['success_tier'], y=success_patterns['avg_tagline_length'], 
                              name='Tagline Length', marker_color='lightyellow'), row=2, col=1)
        fig4.add_trace(go.Bar(x=success_patterns['success_tier'], y=success_patterns['avg_comments'], 
                              name='Comments', marker_color='lightcoral'), row=2, col=2)
        
        fig4.update_layout(height=700, title_text="Success Pattern Analysis", showlegend=False)
        fig4.show()
    else:
        print("⚠️  Insufficient data for success pattern analysis")
    
    # 5. Community Engagement - Most discussed products
    print("\n" + "=" * 60)
    print("5️⃣  COMMUNITY ENGAGEMENT LEADERS")
    print("=" * 60)
    
    engagement = pd.read_sql_query(
        """
        SELECT 
            p.name,
            p.tagline,
            p.votes_count,
            p.comments_count,
            COUNT(DISTINCT c.user_id) as unique_commenters,
            ROUND(CAST(p.comments_count AS FLOAT) / NULLIF(p.votes_count, 0), 3) as comment_to_vote_ratio,
            DATE(p.featured_at) as launch_date
        FROM post_row p
        LEFT JOIN comment_row c ON p.id = c.post_id
        WHERE p.comments_count > 10
            AND p.votes_count > 50
        GROUP BY p.id
        ORDER BY p.comments_count DESC, unique_commenters DESC
        LIMIT 15
    """,
        conn
    )
    
    if not engagement.empty:
        print("\n💬 Most Discussed Products:\n")
        display(engagement)
        
        fig5 = px.scatter(
            engagement,
            x='votes_count',
            y='comments_count',
            size='unique_commenters',
            hover_data=['name', 'tagline', 'launch_date'],
            title='Engagement Patterns: Votes vs Comments',
            labels={'votes_count': 'Votes', 'comments_count': 'Comments'},
            color='comment_to_vote_ratio',
            color_continuous_scale='Turbo'
        )
        fig5.show()
    else:
        print("⚠️  Insufficient engagement data")
    
    conn.close()
    print("\n✅ Advanced analytics complete! 🎉")
    
except Exception as e:
    print(f"❌ Analysis failed: {str(e)}")
    raise
finally:
    try:
        conn.close()
    except:
        pass

## 📈 Visualize Trends

Let's create some visualizations to understand Product Hunt trends better.


In [ ]:
# Interactive visualizations with Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3
import os
from pathlib import Path

print("⏱️  Expected runtime: ~5-10 seconds\n")

try:
    # Connect to database
    db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))
    
    if not db_path.exists():
        print("❌ Database not found! Run the sync cell first.")
        raise FileNotFoundError(f"Database not found at {db_path}")
    
    conn = sqlite3.connect(db_path)
    
    # Posts by day of week with error handling
    print("📅 Analyzing Product Launches by Day of Week...")
    posts_by_day = pd.read_sql_query(
        """
        SELECT 
            CASE CAST(strftime('%w', featured_at) AS INTEGER)
                WHEN 0 THEN 'Sunday'
                WHEN 1 THEN 'Monday'
                WHEN 2 THEN 'Tuesday'
                WHEN 3 THEN 'Wednesday'
                WHEN 4 THEN 'Thursday'
                WHEN 5 THEN 'Friday'
                WHEN 6 THEN 'Saturday'
            END as day_of_week,
            CAST(strftime('%w', featured_at) AS INTEGER) as day_num,
            COUNT(*) as count,
            AVG(votes_count) as avg_votes,
            AVG(comments_count) as avg_comments
        FROM post_row
        WHERE featured_at IS NOT NULL
        GROUP BY day_of_week, day_num
        ORDER BY day_num
    """,
        conn,
    )
    
    if posts_by_day.empty:
        print("⚠️  No data found for visualization. Sync may not have completed.")
    else:
        # Create interactive bar chart with Plotly
        fig1 = px.bar(
            posts_by_day,
            x='day_of_week',
            y='count',
            title='Product Launches by Day of Week',
            labels={'count': 'Number of Products', 'day_of_week': 'Day of Week'},
            color='count',
            color_continuous_scale='Viridis',
            hover_data={'count': True, 'avg_votes': ':.1f', 'avg_comments': ':.1f'}
        )
        fig1.update_layout(
            xaxis_title='Day of Week',
            yaxis_title='Number of Products',
            hovermode='x unified',
            coloraxis_showscale=False
        )
        fig1.show()
    
    # Votes vs Comments correlation
    print("\n📊 Analyzing Engagement Metrics...")
    posts_metrics = pd.read_sql_query(
        """
        SELECT 
            name,
            tagline,
            votes_count,
            comments_count,
            DATE(featured_at) as launch_date
        FROM post_row
        WHERE votes_count > 0 AND comments_count > 0
        ORDER BY votes_count DESC
        LIMIT 1000
    """,
        conn,
    )
    
    if posts_metrics.empty:
        print("⚠️  Insufficient data for correlation analysis.")
    else:
        # Interactive scatter plot with hover info
        fig2 = px.scatter(
            posts_metrics,
            x='votes_count',
            y='comments_count',
            title='Votes vs Comments Correlation (Top 1000 Products)',
            labels={'votes_count': 'Votes Count', 'comments_count': 'Comments Count'},
            hover_data=['name', 'tagline', 'launch_date'],
            color='votes_count',
            size='comments_count',
            color_continuous_scale='Reds',
            opacity=0.6,
            log_x=True,
            log_y=True
        )
        fig2.update_layout(
            hovermode='closest',
            xaxis_title='Votes Count (log scale)',
            yaxis_title='Comments Count (log scale)'
        )
        fig2.show()
    
    # Monthly launch trends
    print("\n📈 Analyzing Launch Trends Over Time...")
    monthly_trends = pd.read_sql_query(
        """
        SELECT 
            strftime('%Y-%m', featured_at) as month,
            COUNT(*) as launches,
            AVG(votes_count) as avg_votes,
            SUM(votes_count) as total_votes
        FROM post_row
        WHERE featured_at IS NOT NULL
        GROUP BY month
        ORDER BY month DESC
        LIMIT 24
    """,
        conn,
    )
    
    if not monthly_trends.empty:
        monthly_trends = monthly_trends.sort_values('month')
        
        # Create dual-axis chart
        fig3 = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig3.add_trace(
            go.Bar(name='Product Launches', x=monthly_trends['month'], y=monthly_trends['launches'],
                   marker_color='lightblue'),
            secondary_y=False,
        )
        
        fig3.add_trace(
            go.Scatter(name='Average Votes', x=monthly_trends['month'], y=monthly_trends['avg_votes'],
                      mode='lines+markers', line=dict(color='red', width=3)),
            secondary_y=True,
        )
        
        fig3.update_layout(
            title='Product Launch Trends (Last 24 Months)',
            hovermode='x unified',
            xaxis_title='Month',
            legend=dict(x=0.01, y=0.99)
        )
        fig3.update_yaxes(title_text="Number of Launches", secondary_y=False)
        fig3.update_yaxes(title_text="Average Votes", secondary_y=True)
        
        fig3.show()
    
    conn.close()
    print("\n✅ Visualizations complete! Hover over charts to explore data interactively.")
    
except sqlite3.Error as e:
    print(f"❌ Database error: {str(e)}")
    print("   The database may be corrupted. Try re-running init and sync.")
    raise
except pd.errors.DatabaseError as e:
    print(f"❌ Query error: {str(e)}")
    print("   Check that the database schema is correct.")
    raise
except Exception as e:
    print(f"❌ Visualization failed: {str(e)}")
    print("   Check that data exists in the database and Plotly is installed.")
    raise
finally:
    # Ensure connection is closed
    try:
        conn.close()
    except:
        pass

# 6️⃣ Export to CSV

Export the database tables to CSV files for easy analysis and sharing.


# 📈 Performance Monitoring Dashboard

Track pipeline performance over time to identify trends and optimize scheduled runs.

In [ ]:
# Performance monitoring and sync history visualization
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
import os
from pathlib import Path
from datetime import datetime, timedelta

print("⏱️  Expected runtime: ~5-10 seconds\n")
print("📊 Analyzing pipeline performance...\n")

try:
    # Load sync history if available
    working_dir = Path("/kaggle/working") if Path("/kaggle/working").exists() else Path.cwd()
    history_file = working_dir / "sync_history.txt"
    
    db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))
    
    if history_file.exists():
        print("=" * 60)
        print("🕐 SYNC HISTORY")
        print("=" * 60)
        
        # Parse sync history
        history_data = []
        with open(history_file, 'r') as f:
            for line in f:
                try:
                    timestamp, duration, exit_code = line.strip().split(',')
                    history_data.append({
                        'timestamp': pd.to_datetime(timestamp),
                        'duration_seconds': float(duration),
                        'duration_minutes': float(duration) / 60,
                        'success': int(exit_code) == 0
                    })
                except:
                    continue
        
        if history_data:
            df_history = pd.DataFrame(history_data)
            df_history['date'] = df_history['timestamp'].dt.date
            
            print(f"\n📅 Total sync runs: {len(df_history)}")
            print(f"✅ Successful: {df_history['success'].sum()}")
            print(f"❌ Failed: {(~df_history['success']).sum()}")
            print(f"⏱️  Average duration: {df_history['duration_minutes'].mean():.1f} minutes")
            print(f"⚡ Fastest run: {df_history['duration_minutes'].min():.1f} minutes")
            print(f"🐌 Slowest run: {df_history['duration_minutes'].max():.1f} minutes")
            
            # Create performance visualization
            fig = make_subplots(
                rows=2, cols=1,
                subplot_titles=('Sync Duration Over Time', 'Daily Success Rate'),
                vertical_spacing=0.15
            )
            
            # Duration trend
            colors = ['green' if s else 'red' for s in df_history['success']]
            fig.add_trace(
                go.Scatter(
                    x=df_history['timestamp'],
                    y=df_history['duration_minutes'],
                    mode='lines+markers',
                    name='Duration',
                    marker=dict(color=colors, size=8),
                    line=dict(color='blue', width=2)
                ),
                row=1, col=1
            )
            
            # Success rate by day
            daily_stats = df_history.groupby('date').agg({
                'success': ['sum', 'count']
            }).reset_index()
            daily_stats.columns = ['date', 'successful', 'total']
            daily_stats['success_rate'] = (daily_stats['successful'] / daily_stats['total'] * 100)
            
            fig.add_trace(
                go.Bar(
                    x=daily_stats['date'],
                    y=daily_stats['success_rate'],
                    name='Success Rate',
                    marker_color='lightgreen'
                ),
                row=2, col=1
            )
            
            fig.update_xaxes(title_text="Time", row=1, col=1)
            fig.update_yaxes(title_text="Duration (minutes)", row=1, col=1)
            fig.update_xaxes(title_text="Date", row=2, col=1)
            fig.update_yaxes(title_text="Success Rate (%)", row=2, col=1)
            
            fig.update_layout(
                height=700,
                title_text="Pipeline Performance Metrics",
                showlegend=True,
                hovermode='x unified'
            )
            
            fig.show()
            
            # Recent runs detail
            print("\n📋 Recent Sync Runs:")
            recent = df_history.tail(10)[['timestamp', 'duration_minutes', 'success']].copy()
            recent['status'] = recent['success'].map({True: '✅ Success', False: '❌ Failed'})
            recent['duration_minutes'] = recent['duration_minutes'].round(2)
            recent = recent[['timestamp', 'duration_minutes', 'status']]
            display(recent)
        else:
            print("⚠️  No valid sync history found")
    else:
        print("⚠️  No sync history file found yet")
        print("   History tracking starts after first sync in this session")
    
    # Database growth metrics
    if db_path.exists():
        conn = sqlite3.connect(db_path)
        
        print("\n" + "=" * 60)
        print("📈 DATA GROWTH METRICS")
        print("=" * 60)
        
        # Monthly growth
        growth = pd.read_sql_query(
            """
            SELECT 
                strftime('%Y-%m', featured_at) as month,
                COUNT(*) as new_posts,
                SUM(COUNT(*)) OVER (ORDER BY strftime('%Y-%m', featured_at)) as cumulative_posts
            FROM post_row
            WHERE featured_at IS NOT NULL
            GROUP BY month
            ORDER BY month DESC
            LIMIT 12
            """,
            conn
        )
        
        if not growth.empty:
            growth = growth.sort_values('month')
            
            # Create growth chart
            fig2 = make_subplots(specs=[[{"secondary_y": True}]])
            
            fig2.add_trace(
                go.Bar(
                    name='New Posts',
                    x=growth['month'],
                    y=growth['new_posts'],
                    marker_color='lightblue'
                ),
                secondary_y=False
            )
            
            fig2.add_trace(
                go.Scatter(
                    name='Cumulative Total',
                    x=growth['month'],
                    y=growth['cumulative_posts'],
                    mode='lines+markers',
                    line=dict(color='red', width=3)
                ),
                secondary_y=True
            )
            
            fig2.update_xaxes(title_text="Month")
            fig2.update_yaxes(title_text="New Posts", secondary_y=False)
            fig2.update_yaxes(title_text="Cumulative Total", secondary_y=True)
            
            fig2.update_layout(
                title_text="Database Growth Trend",
                hovermode='x unified',
                height=400
            )
            
            fig2.show()
            
            print("\n📊 Growth Statistics:")
            display(growth)
        
        # Database file size
        db_size_mb = db_path.stat().st_size / (1024 * 1024)
        print(f"\n💾 Database file size: {db_size_mb:.2f} MB")
        
        # Storage efficiency
        total_records = pd.read_sql_query(
            "SELECT SUM(cnt) as total FROM (SELECT COUNT(*) as cnt FROM post_row UNION ALL SELECT COUNT(*) FROM user_row UNION ALL SELECT COUNT(*) FROM topic_row UNION ALL SELECT COUNT(*) FROM comment_row UNION ALL SELECT COUNT(*) FROM vote_row)",
            conn
        )['total'].iloc[0]
        
        bytes_per_record = (db_size_mb * 1024 * 1024) / total_records if total_records > 0 else 0
        print(f"📦 Storage efficiency: {bytes_per_record:.0f} bytes/record")
        print(f"📝 Total records: {total_records:,}")
        
        conn.close()
    
    print("\n✅ Performance monitoring complete!")
    
except Exception as e:
    print(f"❌ Performance monitoring failed: {str(e)}")
    print("   This is non-critical - pipeline can continue without monitoring")
finally:
    try:
        conn.close()
    except:
        pass

In [ ]:
# Export database to multiple formats with performance comparison
import subprocess
import os
from pathlib import Path
import pandas as pd
import sqlite3
from datetime import datetime

print("⏱️  Expected runtime: ~1-3 minutes\n")

export_dir = Path(os.environ.get('EXPORT_DIR', '/kaggle/working/export'))
db_path = Path(os.environ.get('DB_PATH', '/kaggle/working/producthunt.db'))

try:
    # Export to CSV (default format)
    print("📤 Exporting to CSV format...")
    result = subprocess.run(
        ["producthuntdb", "export"],
        capture_output=True,
        text=True,
        check=False
    )
    
    if result.returncode == 0:
        print("✅ CSV export completed")
        print(result.stdout)
    else:
        print("⚠️  CSV export encountered issues:")
        print(result.stderr)
        if "database is locked" in result.stderr.lower():
            print("\n💡 Database is locked - close other connections and retry")
        raise RuntimeError(f"Export failed: {result.stderr}")
    
    # Additional exports: Parquet and JSON for different use cases
    print("\n📦 Creating additional export formats...")
    
    if not db_path.exists():
        print("⚠️  Database not found, skipping additional exports")
    else:
        conn = sqlite3.connect(db_path)
        
        # Export key tables to Parquet (efficient for data science)
        parquet_dir = export_dir / "parquet"
        parquet_dir.mkdir(parents=True, exist_ok=True)
        
        print("   → Exporting to Parquet (optimized for data analysis)...")
        tables_to_export = ['post_row', 'user_row', 'topic_row', 'comment_row', 'vote_row']
        
        for table in tables_to_export:
            try:
                df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
                if not df.empty:
                    parquet_file = parquet_dir / f"{table}.parquet"
                    df.to_parquet(parquet_file, index=False, compression='snappy')
                    print(f"      ✓ {table}.parquet")
            except Exception as e:
                print(f"      ✗ {table}: {str(e)}")
        
        # Export summary to JSON (lightweight API-friendly format)
        json_dir = export_dir / "json"
        json_dir.mkdir(parents=True, exist_ok=True)
        
        print("   → Exporting summaries to JSON (API-friendly)...")
        
        # Top products summary
        top_products = pd.read_sql_query(
            """
            SELECT 
                id, name, tagline, votes_count, comments_count,
                DATE(featured_at) as featured_date, url
            FROM post_row
            ORDER BY votes_count DESC
            LIMIT 100
            """,
            conn
        )
        if not top_products.empty:
            top_products.to_json(json_dir / "top_products.json", orient='records', indent=2)
            print("      ✓ top_products.json")
        
        # Popular topics summary
        topics_summary = pd.read_sql_query(
            """
            SELECT 
                t.name, t.slug, t.description,
                COUNT(DISTINCT ptl.post_id) as product_count
            FROM topic_row t
            LEFT JOIN post_topic_link ptl ON t.id = ptl.topic_id
            GROUP BY t.id
            ORDER BY product_count DESC
            LIMIT 50
            """,
            conn
        )
        if not topics_summary.empty:
            topics_summary.to_json(json_dir / "popular_topics.json", orient='records', indent=2)
            print("      ✓ popular_topics.json")
        
        conn.close()
    
    # List exported files with size comparison
    print("\n" + "=" * 70)
    print("📁 EXPORTED FILES SUMMARY")
    print("=" * 70)
    
    if export_dir.exists():
        file_stats = []
        
        # CSV files
        for csv_file in sorted(export_dir.glob("*.csv")):
            size_kb = csv_file.stat().st_size / 1024
            file_stats.append({
                'Format': 'CSV',
                'File': csv_file.name,
                'Size (KB)': round(size_kb, 1),
                'Size (MB)': round(size_kb / 1024, 2)
            })
        
        # Parquet files
        if (export_dir / "parquet").exists():
            for parquet_file in sorted((export_dir / "parquet").glob("*.parquet")):
                size_kb = parquet_file.stat().st_size / 1024
                file_stats.append({
                    'Format': 'Parquet',
                    'File': parquet_file.name,
                    'Size (KB)': round(size_kb, 1),
                    'Size (MB)': round(size_kb / 1024, 2)
                })
        
        # JSON files
        if (export_dir / "json").exists():
            for json_file in sorted((export_dir / "json").glob("*.json")):
                size_kb = json_file.stat().st_size / 1024
                file_stats.append({
                    'Format': 'JSON',
                    'File': json_file.name,
                    'Size (KB)': round(size_kb, 1),
                    'Size (MB)': round(size_kb / 1024, 2)
                })
        
        if file_stats:
            stats_df = pd.DataFrame(file_stats)
            print("\n")
            display(stats_df)
            
            # Summary by format
            print("\n📊 Size Comparison by Format:")
            format_summary = stats_df.groupby('Format')['Size (MB)'].agg(['count', 'sum']).round(2)
            format_summary.columns = ['File Count', 'Total Size (MB)']
            display(format_summary)
            
            # Format recommendations
            print("\n💡 Format Recommendations:")
            print("   • CSV: Universal compatibility, human-readable, larger file size")
            print("   • Parquet: 50-80% smaller, fast loading, best for pandas/data science")
            print("   • JSON: API-friendly, smaller summaries, good for web applications")
        else:
            print("⚠️  No files found in export directory")
    else:
        print(f"⚠️  Export directory not found: {export_dir}")
    
    print("\n✅ Export complete!")
    
except FileNotFoundError:
    print("❌ 'producthuntdb' command not found!")
    print("   Re-run the installation cell to fix this.")
    raise
except subprocess.CalledProcessError as e:
    print(f"❌ Export command failed: {str(e)}")
    raise
except Exception as e:
    print(f"❌ Export failed: {str(e)}")
    raise

# 7️⃣ Publish to Kaggle

Publish your dataset to Kaggle! This will create a new dataset or update an existing one.

**Prerequisites:**

1. `KAGGLE_USERNAME` and `KAGGLE_KEY` - Your Kaggle API credentials
2. `KAGGLE_DATASET_SLUG` - Dataset identifier (e.g., `yourusername/product-hunt-database`)

**Setup:**

1. Go to **Notebook Settings** → **Add-ons** → **Secrets**
2. Add the three secrets listed above

<div class="info-card">
    <h3>📝 Note</h3>
    Publishing from a Kaggle notebook to Kaggle may have limitations. For production use, consider running the publish command from a local environment or CI/CD pipeline.
</div>


In [ ]:
# Publish to Kaggle (requires credentials to be configured)
import os
import subprocess

try:
    # Check if credentials are already set from installation cell
    kaggle_username = os.getenv("KAGGLE_USERNAME")
    kaggle_key = os.getenv("KAGGLE_KEY")
    kaggle_slug = os.getenv("KAGGLE_DATASET_SLUG")
    
    if not all([kaggle_username, kaggle_key, kaggle_slug]):
        print("⚠️  Kaggle credentials not configured.")
        print("   Publishing to Kaggle requires:")
        print("   • KAGGLE_USERNAME")
        print("   • KAGGLE_KEY (from kaggle.com/settings)")
        print("   • KAGGLE_DATASET_SLUG (format: username/dataset-name)")
        print("\n   Add these as Kaggle Secrets and re-run the installation cell.")
    else:
        print(f"✅ Publishing to Kaggle dataset: {kaggle_slug}\n")
        
        result = subprocess.run(
            ["producthuntdb", "publish"],
            capture_output=True,
            text=True,
            check=False
        )
        
        if result.returncode == 0:
            print(result.stdout)
            print(f"\n✅ Dataset published successfully!")
            print(f"   View at: https://www.kaggle.com/datasets/{kaggle_slug}")
        else:
            print("⚠️  Publishing encountered issues:")
            print(result.stderr)
            print("\n💡 Troubleshooting:")
            print("   • Verify Kaggle credentials are correct")
            print("   • Ensure dataset exists or CLI can create it")
            print("   • Check you have write permissions")

except Exception as e:
    print(f"❌ Publishing failed: {str(e)}")
    print("   This is optional - core pipeline functionality is not affected.")

# 8️⃣ Schedule Automatic Daily Updates

**This notebook is production-ready for Kaggle's scheduling feature!** Set it up once and your dataset will stay current automatically.

## 🚀 Quick Setup

1. **Complete First Run**: Run all cells once with `--full-refresh` to get historical data
2. **Enable Scheduling**:
   - Click **Notebook** → **Schedule Run**
   - Select **Daily** (or your preferred frequency)
   - Kaggle will run the notebook automatically
3. **That's it!** The default `!producthuntdb sync` in cell 9 is already optimized for incremental updates

## ⚙️ How It Works

The notebook is configured to automatically detect whether it needs a full refresh or incremental update:

- **First run**: Use `--full-refresh` to populate the database
- **Subsequent runs**: Standard `sync` command only fetches new data since last run
- **Safety margins**: Built-in 5-minute lookback prevents data loss
- **Rate limiting**: Automatic retry logic handles API limits gracefully

<div class="info-card">
    <h3>💡 Best Practices</h3>
    <ul style="margin: 0; padding-left: 20px;">
        <li><strong>Initial Setup</strong>: Run with <code>--full-refresh</code> once (cell 9)</li>
        <li><strong>Daily Updates</strong>: Use default <code>sync</code> command (no flags)</li>
        <li><strong>Monitor Logs</strong>: Check execution history for any errors</li>
        <li><strong>Dataset Publishing</strong>: Runs automatically if Kaggle credentials are configured</li>
    </ul>
</div>

## 📊 Expected Performance

| Operation    | Duration    | Data                     |
| ------------ | ----------- | ------------------------ |
| Full Refresh | 2-4 hours   | All historical data      |
| Daily Update | 3-5 minutes | New posts since last run |
| Export       | 1-2 minutes | All tables to CSV        |
| Publish      | 1-2 minutes | Update Kaggle dataset    |

Total scheduled run time: **~10 minutes per day**


# 🎬 Complete Workflow Summary

Here's the complete ProductHuntDB workflow using the CLI:

```bash
# 1. Initialize database
producthuntdb init

# 2. Verify API authentication
producthuntdb verify

# 3. Sync data (choose one)
producthuntdb sync --max-pages 10        # Limited sync (testing)
producthuntdb sync --full-refresh        # Full historical harvest
producthuntdb sync                       # Incremental update

# 4. Check database status
producthuntdb status

# 5. Export to CSV
producthuntdb export

# 6. Publish to Kaggle (requires credentials)
producthuntdb publish

# Advanced: Database migrations
producthuntdb migration-history          # View migration history
producthuntdb migrate "description"      # Create new migration
producthuntdb upgrade head               # Apply migrations
producthuntdb downgrade -1               # Rollback one revision
```

## 📚 CLI Help

For detailed help on any command:

```bash
producthuntdb --help
producthuntdb sync --help
producthuntdb export --help
```


# 🎓 Additional Resources & Troubleshooting

## 📖 Documentation

- **Full Documentation**: [GitHub Repository](https://github.com/wyattowalsh/producthuntdb)
- **API Reference**: [Product Hunt GraphQL API](https://api.producthunt.com/v2/docs)
- **Database Schema**: See `producthuntdb/models.py` for complete schema
- **Configuration Options**: See `producthuntdb/config.py` for all settings

## 🛠️ Troubleshooting Guide

### Common Errors and Solutions

#### 1. Authentication Errors

**Error**: `Authentication failed` or `Invalid token`

**Solutions**:
```python
# Verify token is set correctly
import os
token = os.getenv("PRODUCTHUNT_TOKEN")
print(f"Token configured: {bool(token)}")
print(f"Token length: {len(token) if token else 0} chars")

# Expected: Token configured: True, Length: ~100 chars
```

- Get a new token at: https://api.producthunt.com/v2/oauth/applications
- In Kaggle: Settings → Add-ons → Secrets → Add `PRODUCTHUNT_TOKEN`
- Verify no extra spaces or newlines in token
- Token should start with `"Bearer "` prefix or be the raw OAuth token

#### 2. Database Locked

**Error**: `database is locked` or `OperationalError: database is locked`

**Solutions**:
```bash
# Option 1: Close other connections and retry
# In Python cell:
import sqlite3, os
conn = sqlite3.connect(os.getenv('DB_PATH'))
conn.close()  # Ensure previous connections are closed
```

```bash
# Option 2: Reset database (warning: deletes all data)
!rm -f /kaggle/working/producthunt.db*
!producthuntdb init
```

**Prevention**:
- Always use `try/finally` blocks to close connections
- Avoid running multiple sync operations simultaneously
- Enable WAL mode for better concurrency (automatic in v0.1.0+)

#### 3. Rate Limiting

**Error**: `rate limit exceeded` or `429 Too Many Requests`

**Solutions**:
- Built-in retry logic handles this automatically (exponential backoff)
- Reduce `--max-pages` for testing
- Run sync during off-peak hours (late night/early morning UTC)
- Wait 15-30 minutes between full refresh attempts

**Check rate limit status**:
```python
import subprocess
result = subprocess.run(['producthuntdb', 'verify'], capture_output=True, text=True)
print(result.stdout)  # Shows API status and rate limits
```

#### 4. Memory Errors

**Error**: `MemoryError` or `Out of memory`

**Solutions**:
```python
# Use chunked reading for large queries
import pandas as pd
import sqlite3, os

conn = sqlite3.connect(os.getenv('DB_PATH'))
chunks = []
for chunk in pd.read_sql_query("SELECT * FROM post_row", conn, chunksize=1000):
    # Process chunk
    chunks.append(chunk.head(10))  # Example: keep only top 10 per chunk
result = pd.concat(chunks)
conn.close()
```

**Prevention**:
- Use `LIMIT` clauses in queries
- Enable Parquet export (more memory efficient)
- Clear old notebook outputs: Cell → All Output → Clear

#### 5. Timeout Errors

**Error**: `TimeoutExpired` or sync takes >12 hours

**Solutions**:
- Use incremental sync instead of `--full-refresh`
- Split into smaller batches with `--max-pages 100`
- Run posts-only sync first: `--posts-only`
- Data is saved progressively - re-run to continue

**Monitor progress**:
```bash
# Check what was synced so far
!producthuntdb status
```

#### 6. Import Errors

**Error**: `ModuleNotFoundError: No module named 'producthuntdb'`

**Solutions**:
```bash
# Reinstall package
!pip uninstall -y producthuntdb
!pip install -q git+https://github.com/wyattowalsh/producthuntdb.git

# Verify installation
!python -c "import producthuntdb; print('OK')"
```

#### 7. Database Schema Errors

**Error**: `no such table` or `no such column`

**Solutions**:
```bash
# Check current schema version
!producthuntdb migration-history

# Apply pending migrations
!producthuntdb upgrade head

# If corrupted, reinitialize (warning: deletes data)
!rm -f /kaggle/working/producthunt.db*
!producthuntdb init
```

#### 8. Export Failures

**Error**: Export command fails or produces empty files

**Solutions**:
```python
# Check database has data
import sqlite3, os, pandas as pd
conn = sqlite3.connect(os.getenv('DB_PATH'))
count = pd.read_sql_query("SELECT COUNT(*) as cnt FROM post_row", conn)
print(f"Posts in database: {count['cnt'].iloc[0]}")
conn.close()

# If count = 0, run sync first
```

**Manual export**:
```python
# Export specific table manually
import pandas as pd, sqlite3, os
from pathlib import Path

conn = sqlite3.connect(os.getenv('DB_PATH'))
df = pd.read_sql_query("SELECT * FROM post_row", conn)
export_dir = Path(os.getenv('EXPORT_DIR'))
export_dir.mkdir(parents=True, exist_ok=True)
df.to_csv(export_dir / "post_row.csv", index=False)
print(f"Exported {len(df)} rows to post_row.csv")
conn.close()
```

#### 9. Kaggle Publishing Errors

**Error**: Publishing to Kaggle fails

**Solutions**:
- Verify all three secrets are set:
  - `KAGGLE_USERNAME`
  - `KAGGLE_KEY` (from kaggle.com/settings → API → Create New Token)
  - `KAGGLE_DATASET_SLUG` (format: `username/dataset-name`)
- Dataset must exist on Kaggle first (create manually or let CLI create it)
- Check you have write permissions to the dataset

**Test credentials**:
```python
import os
creds = {
    'KAGGLE_USERNAME': bool(os.getenv('KAGGLE_USERNAME')),
    'KAGGLE_KEY': bool(os.getenv('KAGGLE_KEY')),
    'KAGGLE_DATASET_SLUG': bool(os.getenv('KAGGLE_DATASET_SLUG'))
}
print("Kaggle credentials configured:", all(creds.values()))
for key, val in creds.items():
    print(f"  {key}: {'✅' if val else '❌'}")
```

### Performance Optimization Tips

1. **First Run**: Use `--full-refresh` to get all historical data (2-4 hours)
2. **Daily Updates**: Use default `sync` command (3-5 minutes)
3. **Testing**: Use `--max-pages 10` to sync quickly
4. **Selective Sync**: Use `--posts-only` to skip topics/collections
5. **Monitoring**: Check `sync_history.txt` for performance trends

### Getting Help

**Debug mode** (verbose logging):
```bash
# Enable debug logging (if supported)
!producthuntdb --verbose sync
```

**Check system status**:
```python
import sys, platform, os
from pathlib import Path

print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Working dir: {Path.cwd()}")
print(f"DB Path: {os.getenv('DB_PATH')}")
print(f"Disk free: {shutil.disk_usage('.').free / 1024**3:.1f} GB")
```

**Report issues**:
- **GitHub Issues**: [github.com/wyattowalsh/producthuntdb/issues](https://github.com/wyattowalsh/producthuntdb/issues)
- Include error messages, Python version, and notebook output
- Check existing issues first for known problems

## 🤝 Contributing

Found a bug or have a feature request?

- **Issues**: [GitHub Issues](https://github.com/wyattowalsh/producthuntdb/issues)
- **Pull Requests**: Contributions welcome!

## 📄 License

MIT License - see [LICENSE](https://github.com/wyattowalsh/producthuntdb/blob/main/LICENSE) for details.

# ✅ Production Checklist

Before scheduling this notebook on Kaggle, verify:

- [ ] **Secrets Configured** - `PRODUCTHUNT_TOKEN` is set in Kaggle Secrets
- [ ] **First Run Complete** - Ran once with `--full-refresh` to populate database
- [ ] **Export Works** - CSV files generated successfully
- [ ] **Database Valid** - `producthuntdb status` shows expected data counts
- [ ] **(Optional) Publishing Works** - Kaggle credentials configured and tested

Once checked, you're ready to schedule! 🎉
